In [1]:
import random
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

/home/interaction-lab/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/interaction-lab/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('p8_data_processed.csv')

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y9,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,8.0,1.0,2018-06-07,0.000000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,8.0,1.0,2018-06-07,0.033333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
2,8.0,1.0,2018-06-07,0.066667,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
3,8.0,1.0,2018-06-07,0.100000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
4,8.0,1.0,2018-06-07,0.133333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [4]:
list(df)

['participant',
 'session_num',
 'session_date',
 'timestamp',
 'engagement',
 'activity',
 'skill',
 'difficulty',
 'aptitude',
 'games_total',
 'games_session',
 'mistakes_total',
 'mistakes_session',
 'mistakes_game',
 'ts_robot_talked',
 'ts_game_start',
 'ts_attempt',
 'game_start',
 'game_correct',
 'game_incorrect',
 'mistake_made',
 'ros_PARTICIPANT_STATE',
 'ros_ROBOT_STATE',
 'transcript_spk_0',
 'transcript_spk_1',
 'transcript_spk_2',
 'of_AU01_c',
 'of_AU01_r',
 'of_AU02_c',
 'of_AU02_r',
 'of_AU04_c',
 'of_AU04_r',
 'of_AU05_c',
 'of_AU05_r',
 'of_AU06_c',
 'of_AU06_r',
 'of_AU07_c',
 'of_AU07_r',
 'of_AU09_c',
 'of_AU09_r',
 'of_AU10_c',
 'of_AU10_r',
 'of_AU12_c',
 'of_AU12_r',
 'of_AU14_c',
 'of_AU14_r',
 'of_AU15_c',
 'of_AU15_r',
 'of_AU17_c',
 'of_AU17_r',
 'of_AU20_c',
 'of_AU20_r',
 'of_AU23_c',
 'of_AU23_r',
 'of_AU25_c',
 'of_AU25_r',
 'of_AU26_c',
 'of_AU26_r',
 'of_AU28_c',
 'of_AU45_c',
 'of_AU45_r',
 'of_X_0',
 'of_X_1',
 'of_X_10',
 'of_X_11',
 'of_X_12',
 

In [5]:
# Drop Irrelevant Columns

# transcriptions
df = df.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
df = df.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
df = df.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

df = df.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
df = df.drop(columns=['difficulty', 'skill', 'activity'])

In [6]:
list(df)

['session_num',
 'timestamp',
 'engagement',
 'aptitude',
 'games_total',
 'games_session',
 'mistakes_total',
 'mistakes_session',
 'mistakes_game',
 'ts_robot_talked',
 'ts_game_start',
 'ts_attempt',
 'of_AU01_c',
 'of_AU01_r',
 'of_AU02_c',
 'of_AU02_r',
 'of_AU04_c',
 'of_AU04_r',
 'of_AU05_c',
 'of_AU05_r',
 'of_AU06_c',
 'of_AU06_r',
 'of_AU07_c',
 'of_AU07_r',
 'of_AU09_c',
 'of_AU09_r',
 'of_AU10_c',
 'of_AU10_r',
 'of_AU12_c',
 'of_AU12_r',
 'of_AU14_c',
 'of_AU14_r',
 'of_AU15_c',
 'of_AU15_r',
 'of_AU17_c',
 'of_AU17_r',
 'of_AU20_c',
 'of_AU20_r',
 'of_AU23_c',
 'of_AU23_r',
 'of_AU25_c',
 'of_AU25_r',
 'of_AU26_c',
 'of_AU26_r',
 'of_AU28_c',
 'of_AU45_c',
 'of_AU45_r',
 'of_X_0',
 'of_X_1',
 'of_X_10',
 'of_X_11',
 'of_X_12',
 'of_X_13',
 'of_X_14',
 'of_X_15',
 'of_X_16',
 'of_X_17',
 'of_X_18',
 'of_X_19',
 'of_X_2',
 'of_X_20',
 'of_X_21',
 'of_X_22',
 'of_X_23',
 'of_X_24',
 'of_X_25',
 'of_X_26',
 'of_X_27',
 'of_X_28',
 'of_X_29',
 'of_X_3',
 'of_X_30',
 'of_X_31',

In [8]:
y = df['engagement']

In [9]:
df = df.drop(columns=['engagement'])

In [11]:
X = df.drop(columns=['timestamp'])

In [15]:
list(X)

['session_num',
 'aptitude',
 'games_total',
 'games_session',
 'mistakes_total',
 'mistakes_session',
 'mistakes_game',
 'ts_robot_talked',
 'ts_game_start',
 'ts_attempt',
 'of_AU01_c',
 'of_AU01_r',
 'of_AU02_c',
 'of_AU02_r',
 'of_AU04_c',
 'of_AU04_r',
 'of_AU05_c',
 'of_AU05_r',
 'of_AU06_c',
 'of_AU06_r',
 'of_AU07_c',
 'of_AU07_r',
 'of_AU09_c',
 'of_AU09_r',
 'of_AU10_c',
 'of_AU10_r',
 'of_AU12_c',
 'of_AU12_r',
 'of_AU14_c',
 'of_AU14_r',
 'of_AU15_c',
 'of_AU15_r',
 'of_AU17_c',
 'of_AU17_r',
 'of_AU20_c',
 'of_AU20_r',
 'of_AU23_c',
 'of_AU23_r',
 'of_AU25_c',
 'of_AU25_r',
 'of_AU26_c',
 'of_AU26_r',
 'of_AU28_c',
 'of_AU45_c',
 'of_AU45_r',
 'of_X_0',
 'of_X_1',
 'of_X_10',
 'of_X_11',
 'of_X_12',
 'of_X_13',
 'of_X_14',
 'of_X_15',
 'of_X_16',
 'of_X_17',
 'of_X_18',
 'of_X_19',
 'of_X_2',
 'of_X_20',
 'of_X_21',
 'of_X_22',
 'of_X_23',
 'of_X_24',
 'of_X_25',
 'of_X_26',
 'of_X_27',
 'of_X_28',
 'of_X_29',
 'of_X_3',
 'of_X_30',
 'of_X_31',
 'of_X_32',
 'of_X_33',
 'of

In [17]:
X1 = X.iloc[:,:9]

In [18]:
X1.head()

,session_num,aptitude,games_total,games_session,mistakes_total,mistakes_session,mistakes_game,ts_robot_talked,ts_game_start
0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000108,0.0
2,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000216,0.0
3,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000324,0.0
4,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000432,0.0


### Feature Set 1 - Performance + Caitlyn's Handpicked features

In [26]:
XF1_1 = X[['of_gaze_0_x','of_gaze_0_y',	'of_gaze_0_z',	'of_gaze_1_x',	'of_gaze_1_y',	'of_gaze_1_z',	'of_gaze_angle_x',	'of_gaze_angle_y',	'op_Number of People']]

In [27]:
XF1_1.head()

,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.125
1,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
2,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
3,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
4,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000


In [28]:
XF1 = X1.join(XF1_1)

In [29]:
XF1.head()

,session_num,aptitude,games_total,games_session,mistakes_total,mistakes_session,mistakes_game,ts_robot_talked,ts_game_start,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,op_Number of People
0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.125
1,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000108,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
2,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000216,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
3,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000324,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000
4,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000432,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,0.000


#### Linear SVM - Random 30% Split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(XF1, y, test_size=0.3, random_state=42)

In [32]:
clf = LinearSVC()
clf.fit(X_train,y_train)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [33]:
pred = clf.predict(X_test)
scores = clf.decision_function(X_test)

In [34]:
print("AUC:",roc_auc_score(y_test, scores)) 
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.8888686208640036
Accuracy: 0.8094792321206448
[[28242  7722]
 [ 7135 34882]]
              precision    recall  f1-score   support

         0.0       0.80      0.79      0.79     35964
         1.0       0.82      0.83      0.82     42017

   micro avg       0.81      0.81      0.81     77981
   macro avg       0.81      0.81      0.81     77981
weighted avg       0.81      0.81      0.81     77981



#### Linear SVM - Last 30% Split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(XF1, y, test_size=0.3,shuffle=False)

In [36]:
clf = LinearSVC()
clf.fit(X_train,y_train)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [37]:
pred = clf.predict(X_test)
scores = clf.decision_function(X_test)

In [38]:
print("AUC:",roc_auc_score(y_test, scores)) 
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.7032372654795953
Accuracy: 0.7401931239660943
[[50647  9206]
 [11054  7074]]
              precision    recall  f1-score   support

         0.0       0.82      0.85      0.83     59853
         1.0       0.43      0.39      0.41     18128

   micro avg       0.74      0.74      0.74     77981
   macro avg       0.63      0.62      0.62     77981
weighted avg       0.73      0.74      0.74     77981



### Feature Set 2 - Performance + Selected OP and OF

In [39]:
XF2_1 = X[['of_AU04_c',   'of_AU06_c',    'of_AU07_c',    'of_AU14_c',    'of_AU15_c',    'of_AU25_c',    'of_confidence',    'of_frame',    'of_success',    'of_timestamp', 'op_p1_face_c25',    'op_p1_face_c43',    'op_p1_face_c44',    'op_p1_face_c45',    'op_p1_face_c46',    'op_p1_face_y7',    'op_p1_face_y8',    'op_p1_pose_c0',    'op_p1_pose_c15',    'op_p1_pose_c16']]

In [41]:
XF2_1.head()

,of_AU04_c,of_AU06_c,of_AU07_c,of_AU14_c,of_AU15_c,of_AU25_c,of_confidence,of_frame,of_success,of_timestamp,op_p1_face_c25,op_p1_face_c43,op_p1_face_c44,op_p1_face_c45,op_p1_face_c46,op_p1_face_y7,op_p1_face_y8,op_p1_pose_c0,op_p1_pose_c15,op_p1_pose_c16
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000187,0.000127,0.000112,0.000058,0.000049,0.360333,0.360333,0.162658,0.0,0.457504
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000012,0.0,0.000012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000025,0.0,0.000025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000037,0.0,0.000037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000049,0.0,0.000049,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [42]:
XF2 = X1.join(XF2_1)

#### Linear SVM - Random 30% Split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(XF2, y, test_size=0.3, random_state=42)

In [44]:
clf = LinearSVC()
clf.fit(X_train,y_train)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [45]:
pred = clf.predict(X_test)
scores = clf.decision_function(X_test)

In [46]:
8	70-30 Random
8	70-30 Sessionprint("AUC:",roc_auc_score(y_test, scores)) 
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.9023047245122702
Accuracy: 0.8297662251061156
[[27408  8556]
 [ 4719 37298]]
              precision    recall  f1-score   support

         0.0       0.85      0.76      0.81     35964
         1.0       0.81      0.89      0.85     42017

   micro avg       0.83      0.83      0.83     77981
   macro avg       0.83      0.82      0.83     77981
weighted avg       0.83      0.83      0.83     77981



#### Linear SVM - Last 30% Split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(XF2, y, test_size=0.3,shuffle=False)

In [48]:
clf = LinearSVC()
clf.fit(X_train,y_train)

/home/interaction-lab/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [49]:
pred = clf.predict(X_test)
scores = clf.decision_function(X_test)

In [50]:
print("AUC:",roc_auc_score(y_test, scores)) 
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.8000591943789792
Accuracy: 0.8231620522947898
[[56775  3078]
 [10712  7416]]
              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89     59853
         1.0       0.71      0.41      0.52     18128

   micro avg       0.82      0.82      0.82     77981
   macro avg       0.77      0.68      0.70     77981
weighted avg       0.81      0.82      0.80     77981

